In [76]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, sum, when, lit, year, month, day, date_add, avg, upper, lower, substring, explode, coalesce, pandas_udf
from pyspark.sql.types import StringType, IntegerType, DateType, StructType, StructField
from datetime import datetime, timedelta, date
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import Imputer

spark = SparkSession.builder \
    .appName("SparkReadOperations") \
    .config("spark.executor.cores", "12") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "12") \
    .getOrCreate()

In [77]:
df = spark.read.csv(r'C:\Users\MdMuntasirulHoque\OneDrive - ADA Global\IMPORTANT FILE\POI DATA\IMPORTANT\BD_lookup_table.csv', header=True, inferSchema=True)

In [78]:
df.createOrReplaceTempView("POI")

In [79]:
df.show()

+-------+----------+--------+--------+----------+---------------+
|geohash|   country|division|district|     thana|          union|
+-------+----------+--------+--------+----------+---------------+
| tgzq6c|Bangladesh|  Khulna|Satkhira|Shyamnagar| Satkhira Range|
| tgzqke|Bangladesh|  Khulna|Satkhira|Shyamnagar|   Ramjan Nagar|
| tgzqud|Bangladesh|  Khulna|Satkhira|Shyamnagar|   Ramjan Nagar|
| tgzr0w|Bangladesh|  Khulna|Satkhira|  Kaliganj|       Ratanpur|
| tgzr29|Bangladesh|  Khulna|Satkhira|  Kaliganj|      Dhalbaria|
| tgzrfd|Bangladesh|  Khulna|Satkhira|  Kaliganj|Dakshin Sreepur|
| tgzrne|Bangladesh|  Khulna|Satkhira|Shyamnagar|         Atulia|
| tgzrny|Bangladesh|  Khulna|Satkhira|Shyamnagar|         Atulia|
| tgzrrf|Bangladesh|  Khulna|  Khulna|     Koyra|     Maharajpur|
| tgzrve|Bangladesh|  Khulna|Satkhira|  Assasuni|         Khajra|
| tgzrwe|Bangladesh|  Khulna|Satkhira|  Assasuni|         Anulia|
| tgzx09|Bangladesh|  Khulna|  Khulna|     Koyra|          Koyra|
| tgzx0t|B

In [80]:
spark.sql(
"""
select * from POI

"""
).show(5)

+-------+----------+--------+--------+----------+--------------+
|geohash|   country|division|district|     thana|         union|
+-------+----------+--------+--------+----------+--------------+
| tgzq6c|Bangladesh|  Khulna|Satkhira|Shyamnagar|Satkhira Range|
| tgzqke|Bangladesh|  Khulna|Satkhira|Shyamnagar|  Ramjan Nagar|
| tgzqud|Bangladesh|  Khulna|Satkhira|Shyamnagar|  Ramjan Nagar|
| tgzr0w|Bangladesh|  Khulna|Satkhira|  Kaliganj|      Ratanpur|
| tgzr29|Bangladesh|  Khulna|Satkhira|  Kaliganj|     Dhalbaria|
+-------+----------+--------+--------+----------+--------------+
only showing top 5 rows



In [81]:
spark.sql (
"""
select COUNT(*) FROM POI WHERE division='Dhaka'

"""
).show(5)

+--------+
|count(1)|
+--------+
|   22331|
+--------+



In [82]:
spark.sql (
"""
select * FROM POI WHERE division='Dhaka' and district ='Dhaka'

"""
).show(5)

+-------+----------+--------+--------+----------+---------------+
|geohash|   country|division|district|     thana|          union|
+-------+----------+--------+--------+----------+---------------+
| wh0nkr|Bangladesh|   Dhaka|   Dhaka| Nawabganj|       Galimpur|
| wh0pxw|Bangladesh|   Dhaka|   Dhaka|     Savar|        Biralia|
| wh0q9k|Bangladesh|   Dhaka|   Dhaka|Keraniganj|Subhadya (Part)|
| wh207w|Bangladesh|   Dhaka|   Dhaka|     Savar|       Shimulia|
| wh0nwh|Bangladesh|   Dhaka|   Dhaka|Keraniganj|       Ruhitpur|
+-------+----------+--------+--------+----------+---------------+
only showing top 5 rows



In [83]:
spark.sql (

"""

select division , COUNT(Distinct geohash) as geo 

from POI

GROUP BY division

ORDER BY COUNT(Distinct geohash) DESC 

LIMIT 100
"""


).show(5)

+----------+-----+
|  division|  geo|
+----------+-----+
|Chittagong|25365|
|     Dhaka|22331|
|  Rajshahi|19673|
|    Khulna|18657|
|   Rangpur|17616|
+----------+-----+
only showing top 5 rows



In [84]:
spark.sql (

"""

select COUNT(Distinct division) from POI


Where division ='Dhaka'



"""


"").show(5)

+------------------------+
|count(DISTINCT division)|
+------------------------+
|                       1|
+------------------------+



In [85]:
order=spark.read.csv(r"C:\Users\MdMuntasirulHoque\Downloads\class_3_SQL_IN_SPARK (2)\class_3\ecom\olist_orders_dataset.csv",header=True,inferSchema=True)

In [86]:
order.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [87]:
review=spark.read.csv(r"C:\Users\MdMuntasirulHoque\Downloads\class_3_SQL_IN_SPARK (2)\class_3\ecom\olist_order_reviews_dataset.csv",header=True,inferSchema=True)

In [88]:
review.show()

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                NULL|                  NULL| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                NULL|                  NULL| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                NULL|                  NULL| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                NULL|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [89]:
houseprice=spark.read.csv(r"C:\Users\MdMuntasirulHoque\Downloads\class_3_SQL_IN_SPARK (2)\class_3\data\house_prices.csv", header=True ,inferSchema=True)

In [90]:
houseprice.createOrReplaceTempView("house_prices")
order.createOrReplaceTempView("order")
review.createOrReplaceTempView("review")


In [91]:
houseprice.printSchema()
order.printSchema()
review.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Amount(in rupees): string (nullable = true)
 |-- Price (in rupees): string (nullable = true)
 |-- location: string (nullable = true)
 |-- Carpet Area: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Floor: string (nullable = true)
 |-- Transaction: string (nullable = true)
 |-- Furnishing: string (nullable = true)
 |-- facing: string (nullable = true)
 |-- overlooking: string (nullable = true)
 |-- Society: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Balcony: string (nullable = true)
 |-- Car Parking: string (nullable = true)
 |-- Ownership: string (nullable = true)
 |-- Super Area: string (nullable = true)
 |-- Dimensions: string (nullable = true)
 |-- Plot Area: string (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = t

In [92]:
spark.sql(

"""
select LOWER (Description) as description

from house_prices 
"""
).show(5,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [93]:
spark.sql (

"""

select * from house_prices 


"""

).show(5,truncate=False)

+-----+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------+----------------------+-----------------------------------+--------+-------+-----------+--------------------+----------+----------+---------+
|Index|Title                                                                              |Description                                             

In [94]:
spark.sql (

"""

select * from house_prices 

where Balcony IS NOT null

"""
).show(5,truncate=False)

+-----+-----------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------+----------------------+-----------------------------------+--------+-------+-----------+--------------------+----------+----------+---------+
|Index|Title                                                                              |Description                                                                               

In [95]:
spark.sql (

"""

select * from house_prices 

where Balcony != '2' 

"""
).show(5,truncate=False)

+-----+-----------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------+--------+-----------+-------------+------------+------------+--------------+------------+----------------------+------------------+--------+-------+-----------+--------------------+----------+----------+---------+
|Index|Title                                                                  |Description                                                                                                                 

In [96]:
spark.sql(

"""

select * from house_prices


WHERE Balcony != '2' AND  Balcony !='1'


"""

).show(5)

+-----+--------------------+--------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------------+-----------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+
|Index|               Title|         Description|Amount(in rupees)|Price (in rupees)|location|Carpet Area|       Status|       Floor|Transaction|    Furnishing|      facing|overlooking|             Society|Bathroom|Balcony|Car Parking|           Ownership|Super Area|Dimensions|Plot Area|
+-----+--------------------+--------------------+-----------------+-----------------+--------+-----------+-------------+------------+-----------+--------------+------------+-----------+--------------------+--------+-------+-----------+--------------------+----------+----------+---------+
|   28|3 BHK Ready to Oc...|3 BHK flat availa...|         1.75 Cr |            14344|   thane|   925 sqft|Ready to Move| 8 out of 20|

In [98]:
# Define the condition for checking non-null values
condition = "IS NOT NULL"

# Execute the SQL query using PySpark
spark.sql(
    f"""
    SELECT 
        -- Count the number of non-null values in the 'Title' column
        SUM(CASE WHEN Title {condition} THEN 1 ELSE 0 END) AS Title,

        -- Count the number of non-null values in the 'Description' column
        SUM(CASE WHEN Description {condition} THEN 1 ELSE 0 END) AS Description,

        -- Count the number of non-null values in the 'Amount(in rupees)' column
        SUM(CASE WHEN `Amount(in rupees)` {condition} THEN 1 ELSE 0 END) AS `Amount(in rupees)`,

        -- Count the number of non-null values in the 'Price (in rupees)' column
        SUM(CASE WHEN `Price (in rupees)` {condition} THEN 1 ELSE 0 END) AS `Price (in rupees)`,

        -- Count the number of non-null values in the 'location' column
        SUM(CASE WHEN location {condition} THEN 1 ELSE 0 END) AS location,

        -- Count the number of non-null values in the 'Carpet Area' column
        SUM(CASE WHEN `Carpet Area` {condition} THEN 1 ELSE 0 END) AS `Carpet Area`
    
    FROM house_prices -- Querying data from the 'house_prices' table
    """
).show()  # Display the result in tabular format


+------+-----------+-----------------+-----------------+--------+-----------+
| Title|Description|Amount(in rupees)|Price (in rupees)|location|Carpet Area|
+------+-----------+-----------------+-----------------+--------+-----------+
|187531|     184508|           187531|           169867|  187531|     106861|
+------+-----------+-----------------+-----------------+--------+-----------+



In [99]:
houseprice.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Amount(in rupees): string (nullable = true)
 |-- Price (in rupees): string (nullable = true)
 |-- location: string (nullable = true)
 |-- Carpet Area: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Floor: string (nullable = true)
 |-- Transaction: string (nullable = true)
 |-- Furnishing: string (nullable = true)
 |-- facing: string (nullable = true)
 |-- overlooking: string (nullable = true)
 |-- Society: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Balcony: string (nullable = true)
 |-- Car Parking: string (nullable = true)
 |-- Ownership: string (nullable = true)
 |-- Super Area: string (nullable = true)
 |-- Dimensions: string (nullable = true)
 |-- Plot Area: string (nullable = true)



In [ ]:
#Define Checking NULL 
condition="IS NULL"

# Execute the SQL query using PySpark


spark.sql (

f"""

SELECT  

    SUM(CASE WHEN Title






"""








)